In [21]:
import pandas as pd
import numpy as np
import preprocess
import vectorspace

In [22]:
steam = pd.read_csv("steam_reviews.csv")

In [64]:
steam["helpful_binary"] = steam["helpful"]

,funny,helpful,hour_played
count,4.348910e+05,434891.000000,434891.000000
mean,5.333024e+05,1.004114,364.130773
std,4.785640e+07,59.462935,545.961198
min,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,0.000000,62.000000
50%,0.000000e+00,0.000000,190.000000
75%,0.000000e+00,0.000000,450.000000
max,4.294967e+09,28171.000000,31962.000000


In [23]:
arr = []
for line in steam["review"]:
    line = str(line)
    text = preprocess.removeSGML(line)
    text = preprocess.tokenizeText(text)
    text = preprocess.removeStopwords(text)
    text = preprocess.stemWords(text)
    arr.append(text)

preprocessed = pd.Series(arr)
print(preprocessed)

0         [&gt, Plai, German, Reich&gt, Declar, war, Bel...
1                                                   [ye, .]
2         [Veri, good, game, although, bit, overpr, opin...
3         [Out, review, I, wrote, Thi, on, probabl, most...
4         [Disclaim, I, survivor, main, ., I, plai, game...
                                ...                        
434886    [YOUR, FLESH, WILL, ROT, AND, DECAY, ., STEEL,...
434887    [Domini, Domina, I, believ, we, deal, jewel, ....
434888    [First, off, if, like, X, Com, style, game, mo...
434889    [As, disclaim, I'm, AdMech, player, tabletop, ...
434890    [Don't, listen, peopl, claim, it', game, onli,...
Length: 434891, dtype: object


In [24]:
steam["reviews_processed"] = preprocessed

In [6]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package genesis to
[nltk_data]     /Users/theohaddad/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/theohaddad/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/theohaddad/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/theohaddad/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [58]:
class KNN_Classifer():
    def __init__(self, k=1, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    def predict(self, x_test):
        self.x_test = x_test
        y_predict = []

        for i in range(len(x_test)):
            max_sim = 0
            max_index = 0
            for j in range(len(self.x_train)):
                temp = self.similarity(x_test[i], self.x_train[j])
                if temp > max_sim:
                    max_sim = temp
                    max_index = j
            y_predict.append(self.y_train[max_index])
        return y_predict

    def similarity(self,doc1, doc2):
        synsets1 = self.doc_to_synsets(doc1)
        synsets2 = self.doc_to_synsets(doc2)
        
        return (self.similarity_score(synsets1, synsets2) + self.similarity_score(synsets2, synsets1)) / 2

In [36]:
X_train = list(steam["reviews_processed"][:200000])
Y_train = list(steam["recommendation"][:200000])

X_test = list(steam["reviews_processed"][200001:])
Y_train = list(steam["recommendation"][200001:])

In [61]:
classifier = KNN_Classifer(k=1, distance_type='path')
classifier.fit(X_train, Y_train)
print(classifier.similarity_score)

Y_pred = classifier.predict(X_test)

<bound method KNN_NLC_Classifer.similarity_score of <__main__.KNN_NLC_Classifer object at 0x1d7830790>>


TypeError: can only concatenate list (not "str") to list